# MCLabs Churn Analyzer - Data Preparation

This Jupyter Notebook will be used for general data preparation for the model. As all of our data is coming from separate sources, we first need to combine our data into a singular source. We then will record some values so that we can later derive more features. We will then pre-process the data to address a variety of concerns.